In [1]:
import cv2
import numpy as np
import math

In [8]:
image = cv2.imread('C:\\Users\\987040\\Desktop\\hand.jpg')

#change color space from BGR-->HSV
hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

# performl Thresholding--Create a binary image where white will be skin colors and rest is black
#check: https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_colorspaces/py_colorspaces.html
#you can say that for filtering skin color we have color range [2, 0, 0] to [20, 255, 255]--atleast I could not figure it out
mask2 = cv2.inRange(hsv, np.array([2, 0, 0]), np.array([20, 255, 255]))
#cv2.imshow('Masked Image', mask2)


 
#find contour
#Contours are defined as the line joining all the points along the boundary of an image that are having the same intensity
#findContour() function helps in extracting the contours from the image. 
#It works best on binary images, so we should first apply thresholding techniques
contours,_= cv2.findContours(mask2.copy(),cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

# there would be multiple contours. Find contour with maximum area from contours
#max(contours, key=someFunc())
contour = max(contours, key=lambda x: cv2.contourArea(x))

#--------------OPTIONAL-------------------------------
#cv2.boundingRect() function of OpenCV is used to draw an approximate rectangle around the binary image. 
#x, y, w, h = cv2.boundingRect(contour)   #x, y--coord of top left corner, w,h--width, height of rectangle

#draw this rectangle over image
#cv2.rectangle(image, (x, y), (x + w, y + h), (0, 0, 255), 2)
#cv2.imshow('Contour Rectangle', image)
#--------------OPTIONAL-------------------------------


# Find convex hull--Gift wrapping algorithm
hull = cv2.convexHull(contour)

# Draw contour
drawing = np.zeros(image.shape, np.uint8)  #get a black background
cv2.drawContours(drawing, [contour], -1, (0, 255, 0), 0) #green--contour
cv2.drawContours(drawing, [hull], -1, (0, 0, 255), 0)    #red--Hull
cv2.imshow('Contour and Hull', drawing)


hull = cv2.convexHull(contour,returnPoints = False)
defects = cv2.convexityDefects(contour,hull)

#print(defects)

print("Shape of defects: {}".format(defects.shape))


# Use cosine rule to find angle of the far point from the start and end point i.e. the convex points (the finger
# tips) for all defects
count_defects = 0

for i in range(defects.shape[0]):
    #print("Loop Number: {}".format(i))
    s, e, f, d = defects[i, 0]
    #print("s e f d: {} {} {} {} : ".format(s,e,f,d))
    start = tuple(contour[s][0])
    end = tuple(contour[e][0])
    far = tuple(contour[f][0])
    
    #print("start end far : {} {} {}: ".format(start,end,far))

    #we have 3 points now, calculating triangle sides from it using distance formuala
    a = math.sqrt((end[0] - start[0]) ** 2 + (end[1] - start[1]) ** 2)
    b = math.sqrt((far[0] - start[0]) ** 2 + (far[1] - start[1]) ** 2)
    c = math.sqrt((end[0] - far[0]) ** 2 + (end[1] - far[1]) ** 2)
    
    #we have triangle now, calculating angle created at "far" vertex from cosine formula.
    angle = (math.acos((b ** 2 + c ** 2 - a ** 2) / (2 * b * c)) * 180) / 3.14

    # if angle<=90 draw a circle at the far point
    if angle <= 90:
        #cv2.putText(image, str(count_defects) , far, cv2.FONT_HERSHEY_SIMPLEX, 2,(0,0,0), 2)
        
        #there are cases when "far" point is close to "start" and "end" points. 
        #we want to ignore this scenario. we caculate area of triangle formed by 3 points using Heron's formula
        #and if area is less then some threshold, we ignore this case
        s = 0.5 * ( a + b + c)
        area= math.sqrt( s * (s - a) * (s - b) * (s - c))
        print("Area: {}".format(area))
        
        if(area > 40):
            count_defects += 1
            cv2.circle(image, far, 1, [0, 0, 255], 5)
    cv2.line(image, start, end, [0, 255, 0], 4)
    
    
    
#Print number of fingers
if count_defects == 0:
    cv2.putText(image, "ONE", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 2,(0,0,255),2)
elif count_defects == 1:
    cv2.putText(image, "TWO", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 2,(0,0,255), 2)
elif count_defects == 2:
    cv2.putText(image, "THREE", (5, 50), cv2.FONT_HERSHEY_SIMPLEX, 2,(0,0,255), 2)
elif count_defects == 3:
    cv2.putText(image, "FOUR", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 2,(0,0,255), 2)
elif count_defects == 4:
    cv2.putText(image, "FIVE", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 2,(0,0,255), 2)
else:
    cv2.putText(image, "Not Defined", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 2,(0,0,255), 2)

cv2.imshow('Image', image)

cv2.waitKey(0)
cv2.destroyAllWindows()

Shape of defects: (15, 1, 4)
